<a href="https://colab.research.google.com/github/Hyllite/kohya-trainer/blob/main/Vladmandic_A1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Important links:
- [Vladmandic A1111](https://github.com/vladmandic/automatic),

In [ ]:
#@markdown # Install/Update Vladmandic AUTOMATIC1111 repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import os
import time

if not os.path.exists("/content/gdrive"):
  print('[1;31mGdrive not connected, using colab storage ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /content/gdrive/$mainpth

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/$mainpth/vlad
  %cd /content/gdrive/$mainpth/vlad
  !git clone -q --branch master https://github.com/vladmandic/automatic


# Define the target directory
target_dir = '/content/gdrive/MyDrive/vlad/automatic/extensions-builtin'

# Clone the GitHub repository to the specified directory
!git clone https://github.com/etherealxx/batchlinks-webui "$target_dir/batchlinks-webui"
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete "$target_dir/a1111-sd-webui-tagcomplete"
!wget https://huggingface.co/AnonymousM/Based-mixes/resolve/main/Based65-Final-Mix.safetensors -P /content/gdrive/MyDrive/vlad/automatic/models/Stable-diffusion
!wget https://huggingface.co/NoCrypt/blessed_vae/resolve/main/blessed-fix.vae.pt -P /content/gdrive/MyDrive/vlad/automatic/models/VAE


clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # ControlNet
from torch.hub import download_url_to_file
from urllib.parse import urlparse
import re
from subprocess import run

Model = "None" #@param [ "None", "All (21GB)", "Canny", "Depth", "Lineart", "MLSD", "Normal", "OpenPose", "Scribble", "Seg", "ip2p", "Shuffle", "Inpaint", "Softedge", "Lineart_Anime", "Tile", "T2iadapter_Models"]

#@markdown - Download/update ControlNet extension and its models

def download(url, model_dir):

    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: '+os.path.basename(url))
        download_url_to_file(url, pth, hash_prefix=None, progress=True)
    else:
      print(f"[1;32mThe model {filename} already exists[0m")

Canny='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth'
Depth='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth'
Lineart='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth'
MLSD='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth'
Normal='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth'
OpenPose='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth'
Scribble='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth'
Seg='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth'
ip2p='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth'
Shuffle='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth'
Inpaint='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth'
Softedge='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth'
Lineart_Anime='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth'
Tile='https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth'

with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/vlad/automatic/extensions-builtin
  if not os.path.exists("sd-webui-controlnet"):
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd /content
  else:
    %cd sd-webui-controlnet
    !git reset --hard
    !git pull
    %cd /content

mdldir='/content/gdrive/'+mainpth+'/vlad/automatic/extensions-builtin/sd-webui-controlnet/models'
for filename in os.listdir(mdldir):
  if "_sd14v1" in filename:
    renamed = re.sub("_sd14v1", "-fp16", filename)
    os.rename(os.path.join(mdldir, filename), os.path.join(mdldir, renamed))

!wget -q -O CN_models.txt https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/CN_models.txt

with open("CN_models.txt", 'r') as f:
  mdllnk = f.read().splitlines()

!rm CN_models.txt

with capture.capture_output() as cap:
  cfgnames=[os.path.basename(url).split('.')[0]+'.yaml' for url in mdllnk_v2]
  %cd /content/gdrive/$mainpth/vlad/automatic/extensions-builtin/sd-webui-controlnet/models
  for name in cfgnames:
      run(['cp', 'cldm_v21.yaml', name])
  %cd /content

if Model == "All (21GB)":
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()

elif Model == "T2iadapter_Models":
  mdllnk=list(filter(lambda x: 't2i' in x, mdllnk))
  for lnk in mdllnk:
      download(lnk, mdldir)
  clear_output()

elif Model == "None":
    pass
    clear_output()

else:
  download(globals()[Model], mdldir)
  clear_output()
  inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown #Run Vladmandic A1111
import os

#Launch:
%cd /content/gdrive/$mainpth/vlad/automatic

model_path = "/content/gdrive/MyDrive/vlad/automatic/models" #@param {type:"string"}

#@markdown Insert the full path of your trained model or to a folder containing multiple models

!COMMANDLINE_ARGS="--no-hashing --share --models-dir  {model_path}" REQS_FILE="requirements.txt" python launch.py

In [ ]:
# @title ## **Download Generated Images V2**
import shutil
import os
from IPython.utils import capture
from huggingface_hub import login, HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError
from colablib.colored_print import cprint, print_line

# @markdown Download your output by upload it to **Huggingface** instead of Google Drive.
# @markdown > Get **your** huggingface `WRITE` token [here](https://huggingface.co/settings/tokens)
write_token = ""  # @param {type:"string"}
# @markdown Specify where is your repo located, it will automatically create your repo if you didn't have one.
repo_name = "cagliostro-colab-ui"  # @param{type:"string"}
private_repo = False  # @param{type:"boolean"}
# @markdown This will be compressed to zip and uploaded to datasets repo
project_name = "waifu"  # @param {type :"string"}

repo_name = repo_name.replace(" ", "-")
project_name = project_name.replace(" ", "_")

if not project_name:
    project_name = "waifu"

dataset_zip = f"{project_name}.zip"
output_path = os.path.join(root_dir, dataset_zip)
commit_message = f"Feat: Upload {dataset_zip} with Cagliostro Colab UI"

def create_or_validate_repo(api, datasets_repo):
    try:
        validate_repo_id(datasets_repo)
        api.create_repo(
            repo_id=datasets_repo, repo_type="dataset", private=private_repo
        )
        cprint(f"Repo created, located at "
              f"https://huggingface.co/datasets/{datasets_repo}", color="green")

    except HfHubHTTPError:
        cprint(f"Repo exists, skipping...", color="green")

def compress_to_zip(output_path):
    os.chdir(output_dir)
    cprint(f"Compressing to ZIP...", color="green")
    with capture.capture_output() as cap:
        !zip -rv {output_path} .

def upload_and_cleanup(api, output_path, datasets_repo):
    cprint(f"Uploading generated images... Please wait...", color="green")

    api.upload_file(
        path_or_fileobj=output_path,
        path_in_repo=dataset_zip,
        repo_id=datasets_repo,
        repo_type="dataset",
        commit_message=commit_message,
    )

    cprint(f"Upload success, download directly at "
          f"https://huggingface.co/datasets/{datasets_repo}/resolve/main/{dataset_zip}", color="green")

    os.remove(output_path)

def main():
    with capture.capture_output() as cap:
        login(write_token, add_to_git_credential=True)
    output = cap.stdout.strip()

    if "Token is valid." in output:
        cprint(f"Login Successful.", color="green")

    api = HfApi()
    user = api.whoami(write_token)
    datasets_repo = f"{user['name']}/{repo_name.strip()}"

    if repo_name:
        create_or_validate_repo(api, datasets_repo)
        compress_to_zip(output_path)
        upload_and_cleanup(api, output_path, datasets_repo)

main()
